In [70]:
import requests
import pandas as pd
import numpy as np
import import_ipynb
import ChartJS

In [71]:
import argparse
import sys

from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException




def getData(client, customer_id):
    ga_service = client.get_service("GoogleAdsService", version="v5")

    query = """
        SELECT
          campaign.id,
          campaign.name,
          ad_group.id,
          ad_group.name,
          segments.week,
          segments.device,
          ad_group_criterion.keyword.text,
          ad_group_criterion.keyword.match_type,
          ad_group_criterion.placement.url
          metrics.impressions,
          metrics.clicks,
          metrics.cost_micros,
          metrics.ctr
        FROM keyword_view
        WHERE
          segments.date DURING LAST_MONTH
          AND campaign.advertising_channel_type = 'SEARCH'
          AND ad_group.status = 'ENABLED'
          AND ad_group_criterion.status IN ('ENABLED', 'PAUSED')
        ORDER BY metrics.impressions DESC
        LIMIT 100"""

    # Issues a search request using streaming.
    response = ga_service.search_stream(customer_id, query)
    keyword_match_type_enum = client.get_type(
        "KeywordMatchTypeEnum", version="v5"
    ).KeywordMatchType
    
    all_data=[]
    try:
        for batch in response:
            for row in batch.results:
                single_row = dict()
                single_row['campaign_name'] = row.campaign.name
                single_row['ad_group_name'] = row.ad_group.name
                single_row['keyword'] = row.ad_group_criterion.keyword.text
                single_row['keyword_matchtype'] = row.ad_group_criterion.keyword.match_type
                single_row['impressions'] = row.metrics.impressions
                single_row['clicks'] = row.metrics.clicks
                single_row['cost'] = row.metrics.cost_micros
                print(single_row)
                
                all_data.append(single_row)
    except GoogleAdsException as ex:
        print(
            f'Request with ID "{ex.request_id}" failed with status '
            f'"{ex.error.code().name}" and includes the following errors:'
        )
        for error in ex.failure.errors:
            print(f'\tError with message "{error.message}".')
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print(f"\t\tOn field: {field_path_element.field_name}")
        sys.exit(1)
    df = pd.DataFrame(all_data)
    df.to_csv('google_ads_keywords'.csv)


# if __name__ == "__main__":
#     # GoogleAdsClient will read the google-ads.yaml configuration file in the
#     # home directory if none is specified.
#     googleads_client = GoogleAdsClient.load_from_storage(version="v12")

#     parser = argparse.ArgumentParser(
#         description=("Retrieves a campaign's negative keywords.")
#     )
#     # The following argument(s) should be provided to run the example.
#     parser.add_argument(
#         "-c",
#         "--customer_id",
#         type=str,
#         required=True,
#         help="The Google Ads customer ID.",
#     )
#     args = parser.parse_args()

#     try:
#         main(googleads_client, args.customer_id)
#     except GoogleAdsException as ex:
#         print(
#             f'Request with ID "{ex.request_id}" failed with status '
#             f'"{ex.error.code().name}" and includes the following errors:'
#         )
#         for error in ex.failure.errors:
#             print(f'\tError with message "{error.message}".')
#             if error.location:
#                 for field_path_element in error.location.field_path_elements:
#                     print(f"\t\tOn field: {field_path_element.field_name}")
#         sys.exit(1)

In [72]:
class Main:
    
    
    def get_keyword(self, rows):
        df = pd.DataFrame(index=np.arange(rows), columns=['Campaign_ID', 'Campaign_Name', 'Ad_Group_Name', 'Criterion_ID',
                        'Keyword_Text', 'Match_Type','Device', 'placement_urls','CTR', 'Impressions', 'Clicks', 'Cost_Micros','Week'])

        campaign_id = ['1001', '1002', '1003', '1004']
        df['Campaign_ID'] = np.random.choice(campaign_id, len(df))

        campaign_name = ['Spring Sale', 'Mens Clothing' , 'Winter Clearance', 'Home Decor']
        df["Campaign_Name"] = np.random.choice(campaign_name, len(df))
        df


        Ad_Group_Name =  ["Fashion", "Travel", "Holiday", "Net Banking"]
        df["Ad_Group_Name"] = np.random.choice(Ad_Group_Name, len(df))

        Criterion_ID =  ['20001', '20002', '20003', '20004']

        df["Criterion_ID"] = np.random.choice(Criterion_ID, len(df))

        Keyword_Text =  [
            "Buy designer shoes",
            "Book a luxury hotel in New York",
            "Get cheap flights to Paris",
            "Buy now and save 20%!",
            "Clearance sale - save up to 50%",
            "Free returns and exchanges",
            "Huge selection - shop now",
            "Save big on your favorite brands",
            "24/7 access to your accounts",
            "Sign up for internet banking today"]
        df["Keyword_Text"] = np.random.choice(Keyword_Text, len(df))

        Match_Type = ["BROAD", "EXACT", "PHRASE", "UNKNOWN", "UNSPECIFIED"]
        df["Match_Type"] = np.random.choice(Match_Type, len(df))

        Device = ['CONNECTED_TV', 'DESKTOP','MOBILE', 'OTHER', 'TABLET'
                  'UNKNOWN', 'UNSPECIFIED']
        df["Device"] = np.random.choice(Device, len(df)) 


        placement_urls = ['www.example.com/ad1', 'www.example.com/ad2', 'www.example.com/ad3']
        df['placement_urls'] = np.random.choice(placement_urls, len(df))


        df['CTR'] = np.random.uniform(0, 10, df.shape[0])
        df['Impressions'] = np.random.randint(10000, 50000, df.shape[0])
        df['Clicks'] = np.random.randint(1000, 100000, df.shape[0])
        df['Cost_Micros'] = np.random.randint(1000, 10000, df.shape[0])

        week = ['7/11/2022', '14/11/2022', '21/11/2022', '28/11/2022']
        df["Week"] = np.random.choice(week, len(df))
        
        return df
        # return custResponse
        
        
    def generateCharts(self):
        
        chartsCont = []
        
        # Get the DataFrame generated by get_keyword
        df = self.get_keyword(1000)  # Replace 1000 with the desired number of rows
        
        #Chart 1
        grouped = df.groupby( 'Campaign_Name').sum().reset_index()
        sorted_data = grouped.sort_values(by=['Clicks', 'Cost_Micros'], ascending=False).head(5)
        chart1 = ChartJS.MultiChart(sorted_data['Campaign_Name'], 'Weekly Clicks and Cost per Campaign Over the Last Weeks', 'col', 'Top performing campaigns - sorted by clicks and campaign cost.', 
                                   'It can be used to identify trends and make informed decisions about how to allocate advertising resources in the future.It can make sure they are not overspending on their advertising efforts',
                                   '1', sorted_data['Cost_Micros'], sorted_data['Clicks'])
        chartsCont.append(chart1)
        
        
         #Chart 2
        grouped_key = df.groupby('Keyword_Text').sum().reset_index()
        sorted_data_key = grouped_key.sort_values(by=['Clicks'], ascending=False).head(5)
        chart2 = ChartJS.MultiChart(sorted_data_key['Keyword_Text'], 'Top performing Keyword Text - sorted by clicks and impressions.', 
                                 'Monitor the performance of the most expensive keywords over time to ensure that they continue to have high margins. If the plot shows that a small number of people are seeing the ads but a high percentage of them are clicking on them, you might want to increase the ad budget or expand the targeting to improve performance.',
                                 '2', sorted_data_key['Impressions'], sorted_data_key['Clicks'])
        chartsCont.append(chart2)
        
        #Chart 3
        df1 = df.groupby(['Keyword_Text','Week'])['Cost_Micros'].sum().reset_index() 
        top3 = df1['Keyword_Text'].value_counts().head(3).index.tolist() 
        df2 = df1[df1['Keyword_Text'].isin(top3)]
        chart3 = ChartJS.MultiChart(df2['Week'],  'Most Expensive keywords this week', 'col stacked', 'It displays the total cost of top 3 keywords over a period of time, allowing for comparison between the cost of  top keywords.', 
                                 'Prioritize keywords with a low cost-to-performance ratio and use this data to inform future keyword selection and bidding decisions.',
                                 '3', df2['Keyword_Text'], df2['Cost_Micros'])
        chartsCont.append(chart3)
        
        #Chart 4
        df_grouped = df.groupby('Campaign_Name')['Impressions'].sum().reset_index()
        chart4 = ChartJS.Chart(df_grouped['Campaign_Name'], df_grouped['Impressions'], 'Impressions Distribution by Campaign Name', 'pie', 'It provides a quick visual representation of the relative performance of each campaign.', 
                                 'Focus on the channels that are driving the most impressions and leads.Consider running A/B tests to optimize your campaigns and maximize your ROI.',
                                 '4')
        chartsCont.append(chart4)
        
        #Chart 5
        data = df[["Keyword_Text", "CTR"]]
        data = data.groupby('Keyword_Text').sum().sort_values('CTR', ascending=False).reset_index()
        df_top = data.head(10)
        remaining_sum = data['CTR'][10:].sum()
        df_top.loc[11] = ['Others', remaining_sum]
        chart5 = ChartJS.Chart(df_top['CTR'], df_top['Keyword_Text'], 'Distribution of CTR by Keyword Text', 'pie', 'It provides a quick visual representation of the CTR by top keyword Texts.', 
                                 'Consider using positive keywords to improve the number of clicks. Also Consider adding more keywords to the mix to increase the overall CTR.',
                                 '5')
        chartsCont.append(chart5)
        
        
        #Chart 6
        grouped = df.groupby( 'Device')['Clicks', 'Cost_Micros'].sum().reset_index()
        sorted_data = grouped.sort_values(by=['Clicks', 'Cost_Micros'], ascending=False).head(5)
        chart6 = ChartJS.MultiChart(sorted_data['Device'], 'Performance of Clicks and Cost by Device Type', 'col', 'Top performing device types - sorted by clicks and cost.', 
                                   'Identify which device types are more effective in terms of generating clicks and/or generating a lower cost per click. Experiment with different strategies or tactics for the device types that are underperforming e.g. change from text ads to image ads).',
                                   '6', sorted_data['Cost_Micros'], sorted_data['Clicks'])
        chartsCont.append(chart6)
        
        #Chart 7
        grouped_url = df.groupby('placement_urls')['Clicks', 'Cost_Micros', 'Impressions'].sum().reset_index()
        sorted_data_url = grouped_url.sort_values(by=['Clicks', 'Cost_Micros', 'Impressions'], ascending=False).head(5)
        chart7 = ChartJS.MultiChart(sorted_data_url['placement_urls'], 'Performance of Clicks and Cost by Device Type', 'col', 'Top performing placement URL  - sorted by clicks, cost and impressions.', 
                                   'Consider adding more placement URLs which are similar to the top performing ones. For example: if a fashion site URL is top performing in clicks, invest more in other fashion- related sites in order to reach a similar audience and potentially achieve similar results..',
                                   '7', sorted_data_url['Cost_Micros'], sorted_data_url['Clicks'], sorted_data_url['Impressions'])
        chartsCont.append(chart6)
        
        return chartsCont
        

    def getCharts(self):
        return self.charts
    


In [73]:
main_instance = Main()


In [74]:
main_instance.generateCharts()

C:\Users\Digvijay Katoch\AppData\Local\Temp\ipykernel_20600\1313281995.py:113: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped = df.groupby( 'Device')['Clicks', 'Cost_Micros'].sum().reset_index()
C:\Users\Digvijay Katoch\AppData\Local\Temp\ipykernel_20600\1313281995.py:121: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_url = df.groupby('placement_urls')['Clicks', 'Cost_Micros', 'Impressions'].sum().reset_index()


[{'title': 'Weekly Clicks and Cost per Campaign Over the Last Weeks',
  'type': 'col',
  'overview': 'Top performing campaigns - sorted by clicks and campaign cost.',
  'action': 'It can be used to identify trends and make informed decisions about how to allocate advertising resources in the future.It can make sure they are not overspending on their advertising efforts',
  'chartIdentifier': '1',
  'datasets': [{'label': 'Cost_Micros',
    'data': [1447456, 1463516, 1337635, 1266020]},
   {'label': 'Clicks', 'data': [13150917, 12989631, 11619280, 11149165]}],
  'label': ['Spring Sale', 'Home Decor', 'Mens Clothing', 'Winter Clearance']},
 {'title': 'Top performing Keyword Text - sorted by clicks and impressions.',
  'type': 'Monitor the performance of the most expensive keywords over time to ensure that they continue to have high margins. If the plot shows that a small number of people are seeing the ads but a high percentage of them are clicking on them, you might want to increase the